In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import xarray as xr

In [3]:
from file_processing import get_netcdf_files, get_rhime_outs
from country_totals import get_country_trace, get_x_to_country_mat
from country_totals import get_xr_dummies, sparse_xr_dot, make_quantiles
#!pip install sparse

In [4]:
species = "hfc134a"
min_model_error = 5
post_dir = "/user/home/cv18710/work_shared/paris_total_methane_inversion/hbmcmc_input_output_dec2023/"
files = get_netcdf_files(post_dir+"hfc134a/annual/")

In [5]:
outs = get_rhime_outs(files)

In [6]:
countries = xr.open_dataset("/user/home/cv18710/work_shared/LPDM/countries/country_EUROPE.nc")
countries_ukmo = xr.open_dataset("/user/home/cv18710/openghg_inversions/countries/country-ukmo_EUROPE.nc")

In [7]:
x_to_country_mats = [get_x_to_country_mat(countries, ds, sparse=True) for ds in outs]
x_to_country_mats_ukmo = [get_x_to_country_mat(countries_ukmo, ds, sparse=True) for ds in outs]

In [8]:
country_traces = [get_country_trace(countries, species, hbmcmc_outs=ds, x_to_country=mat) for ds, mat in zip(outs, x_to_country_mats)]

In [9]:
country_traces_ukmo = [get_country_trace(countries_ukmo, species, hbmcmc_outs=ds, x_to_country=mat) for ds, mat in zip(outs, x_to_country_mats_ukmo)]

In [10]:
country_traces = [trace.expand_dims({"time": [out_ds.Ytime.min().values]}) for trace, out_ds in zip(country_traces, outs)]
country_trace_ds = xr.concat(country_traces, dim="time")

In [11]:
country_traces_ukmo = [trace.expand_dims({"time": [out_ds.Ytime.min().values]}) for trace, out_ds in zip(country_traces_ukmo, outs)]
country_trace_ukmo_ds = xr.concat(country_traces_ukmo, dim="time")

# Getting country prior traces

In [12]:
from flux_output_format import get_prior_samples

In [13]:
idatas = [get_prior_samples(ds, min_model_error) for ds in outs]

Sampling: [bc, sigma, x, ymod, ymodbc]
Sampling: [bc, sigma, x, ymod, ymodbc]
Sampling: [bc, sigma, x, ymod, ymodbc]
Sampling: [bc, sigma, x, ymod, ymodbc]
Sampling: [bc, sigma, x, ymod, ymodbc]


In [14]:
prior_x_traces = [idata.prior.x.isel(chain=0).rename({"x_dim_0": "basis_region"}) for idata in idatas]
country_prior_traces = [get_country_trace(countries, species, hbmcmc_outs=ds, x_to_country=mat, x_trace=xtrace) for ds, xtrace, mat in zip(outs, prior_x_traces, x_to_country_mats)]
country_prior_traces_ukmo = [get_country_trace(countries_ukmo, species, hbmcmc_outs=ds, x_to_country=mat, x_trace=xtrace) for ds, xtrace, mat in zip(outs, prior_x_traces, x_to_country_mats_ukmo)]

/user/home/cv18710/acrg/acrg/paris_formatting/country_totals.py:249: UserWarning: rename 'basis_region' to 'basis_region' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  x_trace = x_trace.rename({dim: "basis_region"})


In [15]:
country_prior_traces = [trace.expand_dims({"time": [out_ds.Ytime.min().values]}) for trace, out_ds in zip(country_prior_traces, outs)]
country_prior_trace_ds = xr.concat(country_prior_traces, dim="time")

In [16]:
country_prior_traces_ukmo = [trace.expand_dims({"time": [out_ds.Ytime.min().values]}) for trace, out_ds in zip(country_prior_traces_ukmo, outs)]
country_prior_trace_ukmo_ds = xr.concat(country_prior_traces_ukmo, dim="time")

In [17]:
country_merged_ds = xr.merge([
    country_prior_trace_ds.mean("draw").rename("countryapriori"), 
    make_quantiles(country_prior_trace_ds, sample_dim="draw").rename("qcountryapriori"),
    country_trace_ds.mean("steps").rename("countryapost"), 
    make_quantiles(country_trace_ds).rename("qcountryapost"),
])

country_ukmo_merged_ds = xr.merge([
    country_prior_trace_ukmo_ds.mean("draw").rename("countryapriori"), 
    make_quantiles(country_prior_trace_ukmo_ds, sample_dim="draw").rename("qcountryapriori"),
    country_trace_ukmo_ds.mean("steps").rename("countryapost"), 
    make_quantiles(country_trace_ukmo_ds).rename("qcountryapost"),
])

# Process flux

We can't use the same method (producing flux traces) because they would be massing... 1000 samples for the EUROPE domain would be several gigabytes.

Since the flux is constant on the basis regions, we can compute means and quantiles before mapping them to the original lat/lon domain.

In [18]:
basis_mats = [get_xr_dummies(ds.basisfunctions, cat_dim="basis_region") for ds in outs]

In [19]:
fluxes = [ds.fluxapriori for ds in outs]
x_traces = [ds.xtrace.rename({"nparam": "basis_region"}) for ds in outs]

In [20]:
x_means = [trace.mean("steps") for trace in x_traces]
x_quantiles = [make_quantiles(trace) for trace in x_traces]

In [21]:
flux_means = [sparse_xr_dot(flux * mat, mean) for flux, mean, mat in zip(fluxes, x_means, basis_mats)] 

In [22]:
flux_quantiles = [sparse_xr_dot(flux * mat, quantiles) for flux, quantiles, mat in zip(fluxes, x_quantiles, basis_mats)] 

In [23]:
prior_x_means = [trace.mean("draw") for trace in prior_x_traces]
prior_x_quantiles = [make_quantiles(trace, sample_dim="draw") for trace in prior_x_traces]

In [24]:
prior_flux_means = [sparse_xr_dot(flux * mat, mean) for flux, mean, mat in zip(fluxes, prior_x_means, basis_mats)] 

In [25]:
prior_flux_quantiles = [sparse_xr_dot(flux * mat, quantiles) for flux, quantiles, mat in zip(fluxes, prior_x_quantiles, basis_mats)] 

In [26]:
times = [ds.Ytime.min().values for ds in outs]

In [27]:
prior_flux_mean_ds = xr.concat([da.expand_dims({"time": [time]}) for da, time in zip(prior_flux_means, times)], dim="time")
flux_mean_ds = xr.concat([da.expand_dims({"time": [time]}) for da, time in zip(flux_means, times)], dim="time")

In [28]:
prior_flux_quantile_ds = xr.concat([da.expand_dims({"time": [time]}) for da, time in zip(prior_flux_quantiles, times)], dim="time")
flux_quantile_ds = xr.concat([da.expand_dims({"time": [time]}) for da, time in zip(flux_quantiles, times)], dim="time")

In [29]:
flux_merged_ds = xr.merge([
    prior_flux_mean_ds.rename("fluxapriori"),
    prior_flux_quantile_ds.rename("qfluxapriori"),
    flux_mean_ds.rename("fluxapost"),
    flux_quantile_ds.rename("qfluxapost")
])

# Merge all

In [30]:
#paris_countries = ["IRELAND", "UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IRELAND", "FRANCE", 
#             "BELGIUM", "NETHERLANDS", "GERMANY", "DENMARK", "SWITZERLAND", "AUSTRIA", "ITALY", 
#             "CZECHIA", "POLAND", "HUNGARY", "SLOVAKIA", "NORWAY", "SWEDEN", "FINLAND"]

paris_countries = ["BELGIUM","SWITZERLAND", "AUSTRIA", "ITALY", "NETHERLANDS",
             "CZECHIA", "POLAND", "HUNGARY", "SLOVAKIA","SWEDEN", "FINLAND"]

In [31]:
country_filt = country_merged_ds.country.isin(paris_countries)

In [32]:
country_ukmo_filt = country_ukmo_merged_ds.country.isin(["BENELUX", "RestEU", "SpaPor",
                                                         "IRELAND", "UNITED KINGDOM", 
                                                         "FRANCE","GERMANY","DENMARK",
                                                         "NORWAY"])

In [33]:
country_merged_ds.where(country_filt, drop=True)

<xarray.Dataset>
Dimensions:          (time: 5, ncountries: 11, probs: 4)
Coordinates:
  * time             (time) datetime64[ns] 2018-01-01 2019-01-01 ... 2022-01-01
    country          (ncountries) object 'SWITZERLAND' 'SWEDEN' ... 'AUSTRIA'
    chain            int64 0
  * probs            (probs) float64 0.025 0.159 0.841 0.975
Dimensions without coordinates: ncountries
Data variables:
    countryapriori   (time, ncountries) float64 5.303e+08 ... 6.698e+08
    qcountryapriori  (probs, time, ncountries) float64 3.66e+08 ... 1.199e+09
    countryapost     (time, ncountries) float64 2.166e+08 3.904e+08 ... 5.48e+08
    qcountryapost    (probs, time, ncountries) float64 1.396e+08 ... 9.523e+08

In [34]:
country_final_ds = xr.concat([country_merged_ds.where(country_filt, drop=True), country_ukmo_merged_ds.where(country_ukmo_filt, drop=True)], dim="ncountries")

#country_final_ds = xr.concat([country_merged_ds.where(country_filt, drop=True), country_ukmo_merged_ds.where(country_ukmo_filt, drop=True)], dim="ncountries")

In [35]:
rhime_emissions = xr.merge([flux_merged_ds, country_final_ds])

In [36]:
output_dir = None

#rhime_emissions.to_netcdf(output_dir)